# Phi-4 Mini ONNX کے ساتھ متوازی فنکشن کالنگ کا ٹیوٹوریل

یہ نوٹ بک دکھاتی ہے کہ Phi-4 Mini کو ONNX Runtime GenAI کے ساتھ متوازی فنکشن کالنگ کے لیے کیسے استعمال کیا جائے۔ فنکشن کالنگ ماڈل کو صارف کی درخواستوں کے مطابق بیرونی ٹولز اور APIs کو ذہانت سے استعمال کرنے کی اجازت دیتی ہے۔

## جائزہ

اس ٹیوٹوریل میں آپ سیکھیں گے:
- Phi-4 Mini کو ONNX Runtime GenAI کے ساتھ سیٹ اپ کرنا
- فلائٹس اور ہوٹلز بک کرنے کے لیے فنکشن اسکیمے کی تعریف کرنا
- ساختہ آؤٹ پٹ کے لیے Lark گرامر کے ساتھ گائیڈڈ جنریشن استعمال کرنا
- پیچیدہ سفر بکنگ کے منظرناموں کے لیے متوازی فنکشن کالز انجام دینا

## ضروریات

اس نوٹ بک کو چلانے سے پہلے، یقینی بنائیں کہ آپ نے:
- Phi-4 Mini ONNX ماڈل ڈاؤنلوڈ کیا ہے
- `onnxruntime-genai` پیکیج انسٹال کیا ہے
- فنکشن کالنگ کے تصورات کی بنیادی سمجھ حاصل کی ہے


## مرحلہ 1: ضروری لائبریریاں درآمد کریں

سب سے پہلے، ہم اپنی فنکشن کالنگ کے نفاذ کے لیے ضروری لائبریریاں درآمد کریں گے۔


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## مرحلہ 2: ماڈل سیٹ اپ اور ترتیب

اب ہم Phi-4 Mini ONNX ماڈل کو ترتیب دیں گے۔ یقینی بنائیں کہ راستہ کو اپنے اصل ماڈل ڈائریکٹری سے تبدیل کریں۔


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## مرحلہ 3: جنریشن پیرامیٹرز کو ترتیب دیں

ماڈل کے آؤٹ پٹ کے رویے کو کنٹرول کرنے کے لیے جنریشن پیرامیٹرز کو ترتیب دیں۔ یہ ترتیبات فنکشن کالنگ کے لیے یقینی اور مرکوز جوابات فراہم کرتی ہیں۔


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## مرحلہ 4: دستیاب فنکشنز کی وضاحت کریں

یہاں ہم ان فنکشنز کی وضاحت کرتے ہیں جنہیں ہمارا AI معاون استعمال کر سکتا ہے۔ اس مثال میں، ہمارے پاس دو سفر سے متعلق فنکشنز ہیں:
1. **booking_flight_tickets**: ہوائی اڈوں کے درمیان پروازوں کی بکنگ کے لیے
2. **booking_hotels**: ہوٹل کی رہائش کی بکنگ کے لیے

فنکشن کی وضاحتیں OpenAI کے فنکشن کالنگ اسکیم فارمیٹ کی پیروی کرتی ہیں۔


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## مرحلہ 5: گرامر بنانے کے معاون فنکشنز

یہ معاون فنکشنز ہماری فنکشن کی تعریفوں کو Lark گرامر فارمیٹ میں تبدیل کرتے ہیں، جو ONNX Runtime GenAI کے ذریعے رہنمائی شدہ تخلیق کے لیے استعمال ہوتا ہے۔ یہ یقینی بناتا ہے کہ ماڈل درست JSON فارمیٹ میں موزوں فنکشن کالز تیار کرے۔


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## مرحلہ 6: گرامر جنریشن کی جانچ کریں

آئیے اپنی گرامر جنریشن فنکشنز کی جانچ کریں تاکہ دیکھ سکیں کہ وہ ہمارے ٹول کی تعریفوں کو صحیح فارمیٹ میں کیسے تبدیل کرتے ہیں۔


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## مرحلہ 7: سسٹم پرامپٹ اور جنریٹر تیار کریں

اب ہم سسٹم پرامپٹ تیار کریں گے جو ماڈل کو دستیاب ٹولز کے بارے میں بتائے گا اور جنریٹر کو رہنمائی شدہ جنریشن پیرامیٹرز کے ساتھ ترتیب دیں گے۔


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## مرحلہ 8: گائیڈڈ جنریشن کے ساتھ جنریٹر کو شروع کریں

اب ہم اپنے ترتیب دیے گئے پیرامیٹرز کے ساتھ جنریٹر بنائیں گے اور گائیڈڈ جنریشن کے لیے Lark گرامر کا اطلاق کریں گے۔


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## مرحلہ 9: متوازی فنکشن کالنگ کی جانچ کریں

اب آئیے ایک پیچیدہ سفر بکنگ کے منظرنامے کے ساتھ اپنے سیٹ اپ کی جانچ کرتے ہیں جس میں متعدد فنکشنز کو کال کرنا ضروری ہے۔


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### فنکشن کالز بنائیں

ماڈل اب صارف کی درخواست کے مطابق منظم فنکشن کالز تیار کرے گا۔ رہنمائی شدہ جنریشن کی بدولت، آؤٹ پٹ درست JSON فارمیٹ میں ہوگا جو براہ راست عمل میں لایا جا سکتا ہے۔

**متوقع نتیجہ**: ماڈل کو درج ذیل فنکشن کالز تیار کرنی چاہئیں:
1. `booking_flight_tickets` بیجنگ (PEK) سے پیرس (CDG) کی تفصیلات کے ساتھ
2. `booking_hotels` پیرس میں رہائش کی تفصیلات کے ساتھ

نیچے دیے گئے سیل کو چلائیں تاکہ لائیو جنریشن دیکھ سکیں:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## نتیجہ

🎉 **مبارک ہو!** آپ نے کامیابی کے ساتھ Phi-4 Mini کے ساتھ ONNX Runtime GenAI کا استعمال کرتے ہوئے متوازی فنکشن کالنگ کو نافذ کر لیا ہے۔

### آپ نے کیا سیکھا:

1. **ماڈل سیٹ اپ**: ONNX Runtime GenAI کے ساتھ Phi-4 Mini کو ترتیب دینے کا طریقہ
2. **فنکشن کی تعریف**: AI فنکشن کالنگ کے لیے فنکشن اسکیموں کی وضاحت کرنے کا طریقہ
3. **رہنمائی شدہ جنریشن**: منظم آؤٹ پٹ جنریشن کے لیے Lark گرامر کا استعمال کرنے کا طریقہ
4. **متوازی فنکشن کالز**: پیچیدہ درخواستوں کو سنبھالنے کا طریقہ جن میں متعدد فنکشن کالز کی ضرورت ہوتی ہے

### اہم فوائد:

- ✅ **منظم آؤٹ پٹ**: رہنمائی شدہ جنریشن درست JSON فنکشن کالز کو یقینی بناتی ہے
- ✅ **متوازی پروسیسنگ**: ایک ہی درخواست میں متعدد فنکشن کالز کو سنبھالنا
- ✅ **اعلی کارکردگی**: ONNX Runtime بہتر انفرنس فراہم کرتا ہے
- ✅ **لچکدار اسکیمہ**: فنکشن کی تعریفوں کو آسانی سے شامل یا تبدیل کرنا

### وسائل:

- [Phi-4 Mini دستاویزات](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI دستاویزات](https://onnxruntime.ai/docs/genai/)
- [فنکشن کالنگ کے بہترین طریقے](https://platform.openai.com/docs/guides/function-calling)



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
